# Web Search Agent Testing Notebook

This notebook walks through testing your Bing-grounded Web Search Agent (created in the Web Search Agent Creation notebook) using the `/agent/stream` endpoint.

Workflow:
1. Ensure the API is running locally
2. Load environment variables (including `AGENT_ID` of the web search agent)
3. Create a thread (`/thread/create`)
4. Send a streaming question to `/agent/stream`
5. Render incremental output live

We'll ask:
> how much did Jaws make at the global box office?

Run cells in order.

## 1. Run the API Locally (Uvicorn)
Ensure the FastAPI server is running (auto-loads `api/.env`). From repo root:
```powershell
python -m venv .venv
.\.venv\Scripts\Activate.ps1
pip install -r api/requirements.txt
uvicorn api.main:app --reload --host 0.0.0.0 --port 8000
```
Open docs: http://localhost:8000/docs
Proceed once the server is running.

## 2. Load Environment & Define Base URL
Loads `.env` and validates required values.

In [32]:
# 2. Load environment & base URL
import os
from dotenv import load_dotenv

load_dotenv('../api/.env', override=True)
BASE_URL = os.getenv('API_BASE_URL', 'http://localhost:8000')
AGENT_ID = os.getenv('WEB_SEARCH_AGENT_ID')
if not AGENT_ID:
    raise ValueError('AGENT_ID not set. Create the web search agent and set AGENT_ID in .env.')
print('Base URL:', BASE_URL)
print('Agent ID:', AGENT_ID)

Base URL: http://localhost:8000
Agent ID: asst_v7qdDNDKV6a6gUM0rbazVKVz


## 3. Create a Thread
Call `/thread/create` to obtain a `thread_id` for this interaction.

In [33]:
# 3. Create thread
import requests
resp = requests.post(f"{BASE_URL}/thread/create")
resp.raise_for_status()
thread_id = resp.json()['thread_id']
print('Thread ID:', thread_id)

Thread ID: thread_g9Vd3r0lMXSSOVhY935mYNTi


## 4. Stream a Web Search Query
Use `/agent/stream` to send the question and render incremental output. The question:
```
how much did Jaws make at the global box office?
```

In [34]:
# 4. Stream query response (inline citation replacement during stream)
import requests
import json
from IPython.display import display, Markdown, clear_output

question = "how much did Jaws make at the global box office?"
url = f"{BASE_URL}/agent/stream"

payload = {
    "thread_id": thread_id,
    "message": question,
    "agent_id": AGENT_ID
}

accumulated = ""
citations_processed = False
processed_text = None  # Holds text after inline citation substitution

with requests.post(url, json=payload, stream=True) as r:
    r.raise_for_status()
    for chunk in r.iter_content(chunk_size=None):
        if not chunk:
            continue
        try:
            text = chunk.decode('utf-8', errors='ignore')
        except Exception:
            continue
        accumulated += text

        # Attempt inline citation processing once citation JSON appears and is parseable
        if not citations_processed and ('##CITATIONS:' in accumulated or '##CITATIONS' in accumulated):
            # Prefer the variant with colon if present
            split_token = '##CITATIONS:' if '##CITATIONS:' in accumulated else '##CITATIONS'
            main_text, citation_blob = accumulated.split(split_token, 1)
            citation_blob = citation_blob.strip()
            try:
                citations = json.loads(citation_blob)
                updated_text = main_text.rstrip()
                # Inline replacement logic
                for citation in citations:
                    try:
                        url_citation = citation.get('url_citation') or {}
                        title = url_citation.get('title') or url_citation.get('url') or 'Source'
                        url = url_citation.get('url') or ''
                        original_text = citation.get('text') or ''
                        if original_text:
                            formatted_citation = f" [{title}]({url}) "
                            updated_text = updated_text.replace(original_text, formatted_citation)
                    except Exception:
                        continue
                processed_text = updated_text
                citations_processed = True
            except Exception:
                # JSON likely incomplete; wait for more chunks
                pass

        clear_output(wait=True)
        if citations_processed and processed_text is not None:
            display(Markdown(f"### Streaming Response (in-progress, citations applied)\n\n**Question:** {question}\n\n{processed_text}"))
        else:
            display(Markdown(f"### Streaming Response (in-progress)\n\n**Question:** {question}\n\n{accumulated}"))

# Final render
clear_output(wait=True)
final_output = processed_text if (citations_processed and processed_text is not None) else accumulated
# If any trailing citation JSON remained, strip it for cleanliness
if '##CITATIONS' in final_output:
    final_output = final_output.split('##CITATIONS')[0].rstrip()

display(Markdown(f"### Final Answer\n\n**Question:** {question}\n\n{final_output}"))
print("--- Stream complete ---")

### Final Answer

**Question:** how much did Jaws make at the global box office?

The global box office earnings for the 1975 film *Jaws* totaled approximately $489.5 million, including $278.8 million from North America and $210.6 million internationally. This figure includes revenue from several re-releases [Jaws - Box Office Mojo](https://www.boxofficemojo.com/title/tt0073195/)  [Jaws Re-Release Box Office: Steven Spielberg’s Classic Hits ... - Koimoi](https://www.koimoi.com/box-office/jaws-re-release-box-office-steven-spielbergs-classic-hits-10m-domestically-for-50th-anniversary/) .<br/>

--- Stream complete ---
